In [1]:
import pandas as pd

import json
import warnings
warnings.filterwarnings("ignore")
import funciones as func
import re
import ast
import pyarrow



In [2]:
archivo = 'australian_user_reviews.json'
# Se lee de cada línea del dataset
fs = []
with open(archivo, encoding = 'utf-8') as f:
    for line in f.readlines():
        fs.append(ast.literal_eval(line))

# Se convierte en dataframe
df = pd.DataFrame(fs)


# Guardo como csv para revisarlo en otra aplicación (LibreOffice Calc)
df.to_csv('reviews.csv', sep=',', index=False, encoding='utf-8')
df.shape


(25799, 3)

In [3]:
porcentaje_nulos= ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
print(porcentaje_nulos)

user_id     0.0
user_url    0.0
reviews     0.0
dtype: float64


In [4]:
# Tipo de dato de user_id
print(df['user_id'].dtype)
# df['user_id'] = df['user_id'].astype(str)

object


In [5]:
# Eliminar duplicados
df_c = df
df_c.drop_duplicates(subset='user_id', inplace=True, keep='first')
df_c.reset_index(drop=True, inplace=True)
df_c.tail()
#df.shape


,user_id,user_url,reviews
25480,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25481,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25482,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25483,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."
25484,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"[{'funny': '1 person found this review funny',..."


In [6]:
# Vemos la estructura de los registros de la columna 'reviews'
df_c.loc[0, 'reviews']

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [7]:
# Lo transformamos en columnas
df_b = pd.json_normalize(df_c['reviews'])
df_b.head()
df_b.shape


(25485, 10)

In [8]:
# A df_b le falta el user_id y el user_url. Sólo recuperamos user_id, que nos sirve.

# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
df_a= pd.concat([df[['user_id']], df_b], axis=1)
df_a.reset_index(drop=True, inplace=True)
df_a.head(10)
#df_completo.shape

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
5,Wackky,"{'funny': '', 'posted': 'Posted May 5, 2014.',...","{'funny': '', 'posted': 'Posted December 24, 2...","{'funny': '1 person found this review funny', ...","{'funny': '', 'posted': 'Posted March 20, 2012...","{'funny': '', 'posted': 'Posted March 9, 2012....",None,None,None,None,None
6,76561198079601835,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
7,MeaTCompany,"{'funny': '', 'posted': 'Posted July 24.', 'la...",None,None,None,None,None,None,None,None,None
8,76561198089393905,"{'funny': '5 people found this review funny', ...","{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None
9,76561198156664158,"{'funny': '', 'posted': 'Posted June 16.', 'la...",None,None,None,None,None,None,None,None,None


In [9]:
# Se utiliza pd.melt para transformar las columnas en filas. Conservo el 'user_id'.
df_1 = pd.melt(df_a, id_vars=['user_id'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_1.head()

,user_id,variable,reviews
0,76561197970982479,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,0,"{'funny': '3 people found this review funny', ..."


In [10]:
df_1.tail()

,user_id,variable,reviews
229360,76561198306599751,8,None
229361,Ghoustik,8,None
229362,76561198310819422,8,None
229363,76561198312638244,8,None
229364,LydiaMorley,8,None


In [11]:
# Dado que no todos los usuarios hicieron la misma cantidad de reviews, se agregaron muchas filas sin reviews
# Las eliminamos
df_2 = df_1.dropna()
# Indexamos nuevamente
df_2.reset_index(drop=True, inplace=True)
df_2.tail(10)


,user_id,variable,reviews
58158,vivatheplayer,8,"{'funny': '', 'posted': 'Posted December 7, 20..."
58159,76561198118697312,8,"{'funny': '', 'posted': 'Posted December 20, 2..."
58160,sh06un1,8,"{'funny': '', 'posted': 'Posted February 3, 20..."
58161,76561198133319761,8,"{'funny': '', 'posted': 'Posted June 27, 2014...."
58162,76561198138691719,8,"{'funny': '', 'posted': 'Posted December 30, 2..."
58163,SKELETRONPRIMEISOP,8,"{'funny': '', 'posted': 'Posted August 15, 201..."
58164,76561198141079508,8,"{'funny': '', 'posted': 'Posted August 2, 2014..."
58165,ShadowYT100,8,"{'funny': '', 'posted': 'Posted July 31, 2015...."
58166,bestcustomurlevermade,8,"{'funny': '', 'posted': 'Posted December 20, 2..."
58167,76561198209894493,8,"{'funny': '', 'posted': 'Posted February 3.', ..."


In [12]:
# Ahora convertimos cada clave del diccionario en una columna
df_3 = df_2['reviews'].apply(pd.Series, dtype='object')
df_3.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [13]:
# Concatenamos con user_id.
df_4 = pd.concat([df_2[['user_id']], df_3], axis=1)
df_4.tail()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
58163,SKELETRONPRIMEISOP,,"Posted August 15, 2014.","Last edited November 3, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...
58164,76561198141079508,,"Posted August 2, 2014.",,304930,No ratings yet,True,Fun game with friends
58165,ShadowYT100,,"Posted July 31, 2015.",,265630,No ratings yet,True,So Fun!! :D
58166,bestcustomurlevermade,,"Posted December 20, 2015.",,304050,No ratings yet,True,"This game is great. The only thing is,Why cant..."
58167,76561198209894493,,Posted February 3.,,570,0 of 2 people (0%) found this review helpful,False,DOTA 2.normal skill - 2ez for me>high skill - ...


In [14]:
# No utilizaremos las columnas 'funny' y 'last_edited'. Las eliminamos.
df_5 = df_4.drop(columns=['funny', 'last_edited'])
df_5.columns

Index(['user_id', 'posted', 'item_id', 'helpful', 'recommend', 'review'], dtype='object')

In [15]:

# Creamos la columna 'date' aplicando la función YMD a la columna 'posted'
# YMD intenta cambiar las fechas del tipo 'December 20, 2015' a 2015-12-20. Si originalmente tiene otro formato o existen datos faltantes, devuelve 'Formato inválido'
df_5['date'] = df_5['posted'].apply(func.YMD)
df_5['date']


0              2011-11-05
1              2014-06-24
2        Formato inválido
3              2013-10-14
4              2014-04-15
               ...       
58163          2014-08-15
58164          2014-08-02
58165          2015-07-31
58166          2015-12-20
58167    Formato inválido
Name: date, Length: 58168, dtype: object

In [16]:
# Ahora nos quedaremos sólo con el año del posteo
df_5['posted_year'] = df_5['date'].apply(func.anio)
df_5['posted_year']

0        2011
1        2014
2           0
3        2013
4        2014
         ... 
58163    2014
58164    2014
58165    2015
58166    2015
58167       0
Name: posted_year, Length: 58168, dtype: object

In [17]:
# Analizando el dataset vemos que en los casos de 'Formato inválido' la fecha no tiene año. No eliminaremos esos registros pues las otras columnas pueden aportar información que puede ser necesaria más adelante.
df_5['posted']

0         Posted November 5, 2011.
1            Posted June 24, 2014.
2               Posted February 3.
3         Posted October 14, 2013.
4           Posted April 15, 2014.
                   ...            
58163      Posted August 15, 2014.
58164       Posted August 2, 2014.
58165        Posted July 31, 2015.
58166    Posted December 20, 2015.
58167           Posted February 3.
Name: posted, Length: 58168, dtype: object

In [18]:
# Ahora nos quedamos solo con la columna 'posted_year'


In [19]:
# Eliminamos 'posted' y 'date'
df_5 = df_5.drop(['posted', 'date'], axis=1)

In [20]:
# Eliminamos espacios vacíos '' por nulos
df_5.replace('', None, inplace=True)
df_5.head()
porcentaje_nulos= ((df_5.isnull().sum() / len(df_5)) * 100).sort_values(ascending=False)
print(porcentaje_nulos)

review         0.051575
user_id        0.000000
item_id        0.000000
helpful        0.000000
recommend      0.000000
posted_year    0.000000
dtype: float64


In [21]:
# Eliminamos los nulos en review
df_6 = df_5.dropna(subset=['review'])
# Verificamos
porcentaje_nulos= ((df_6.isnull().sum() / len(df_6)) * 100).sort_values(ascending=False)
print(porcentaje_nulos)

user_id        0.0
item_id        0.0
helpful        0.0
recommend      0.0
review         0.0
posted_year    0.0
dtype: float64


In [22]:
# Tipo de dato de user_id
print(df_6['user_id'].dtype)
# Convertimos todos los registros a formato string, pues algunos están como números
df_6['user_id'] = df_6['user_id'].astype(str)

object


In [23]:
df_6

,user_id,item_id,helpful,recommend,review,posted_year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
2,evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,0
3,doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013
4,maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014
...,...,...,...,...,...,...
58163,SKELETRONPRIMEISOP,440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2014
58164,76561198141079508,304930,No ratings yet,True,Fun game with friends,2014
58165,ShadowYT100,265630,No ratings yet,True,So Fun!! :D,2015
58166,bestcustomurlevermade,304050,No ratings yet,True,"This game is great. The only thing is,Why cant...",2015


In [26]:
# Elimino las filas donde posted_year = 0 y cambio el tipo de posted_year a int
df_7 = df_6.drop(df_6[df_6['posted_year'] == 0].index)
df_7.reset_index(drop=True, inplace=True)
df_7['posted_year'] = df_7['posted_year'].astype(int)


In [28]:
# Grabamos en CSV el dataset limpio
archivo = 'user_review_final.csv'
df_6.to_csv(archivo, index=False)
# Pasamos a parquet para usarlo en la API
df_7.to_parquet('user_review_final.parquet', compression='gzip')

In [ ]:
# Verificamos la grabación del CSV
dfprueba = pd.read_csv('user_review_final.csv')
dfprueba

,user_id,item_id,helpful,recommend,review,posted_year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
2,doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013
3,maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014
4,Wackky,249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014
...,...,...,...,...,...,...
48221,76561198138691719,332310,No ratings yet,True,Normally I would hardly play a lego based game...,2015
48222,SKELETRONPRIMEISOP,440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2014
48223,76561198141079508,304930,No ratings yet,True,Fun game with friends,2014
48224,ShadowYT100,265630,No ratings yet,True,So Fun!! :D,2015
